In [ ]:
hot_T = pd.DataFrame({'temp':temp_hot}, index=time) rolls_H = hot_T.rolling('300s').std()
#print(rolls) plot_H = rolls_H.plot(title='Hot blackbody rolling standard deviation',fontsize=14, figsize=[16,5], grid=True, legend=False) plot_H.title.set_size(20) plt.savefig('Standard_deviation_hot.png')
cold_T = pd.DataFrame({'temp':temp_cold}, index=time) rolls_C = cold_T.rolling('300s').std()
#print(rolls) plot_C = rolls_C.plot(title='Cold blackbody rolling standard deviation', fontsize=14, figsize=[16,5], grid=True, legend=False) plot_C.title.set_size(20) plt.savefig('Standard_deviation_cold.png')

In [ ]:
rolls_H.reset_index().values[10,1] #reset_index converts it into a Python array

In [ ]:
def func(num):
    temp_hot_slope = []
    temp_cold_slope = []
    for i in range(len(new_temp_hot)-num):
        temp_hot_slope.append((new_temp_hot[i+num]-new_temp_hot[i]))
        temp_cold_slope.append((new_temp_cold[i+num]-new_temp_cold[i]))

    lags = np.arange(-len(temp_hot_slope) + 1, len(temp_hot_slope))
    ccov = np.correlate(temp_hot_slope - np.mean(temp_hot_slope), temp_cold_slope - np.mean(temp_cold_slope), mode='full') #full means all points where the two functions overlap are used. Therefore boundary effects are visible
    ccor = ccov / (len(temp_hot_slope) * np.std(temp_hot_slope) * np.std(temp_cold_slope))
    maxlag = lags[np.argmax(ccor)]
    return(maxlag*2.1/60)

list1 = []

for n in range(3000):
    list1.append(func(n))

plt.scatter(np.arange(3000),list1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

def plot_3d_fns_0(start, stop, azimuth=None, elevation=None): #start and stop is roughly the time in seconds
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(wns_last, rad_last, temps_last, cmap='inferno')


plot_deltas(0, wns_last[-1])

In [ ]:
def plot_3d_fn(start, stop, azimuth=None, elevation=None): #start and stop is roughly the time in seconds
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(wns_first, temps_first, rad_first, cmap='inferno')
    ax.set_xlabel('Wavenumber')
    ax.set_ylabel('Temperature')
    ax.set_zlabel('Radiance')

plot_3d_fn(0, wns_first[-1])

In [ ]:
temps_avg = []
temps_std = []

for (spectrum, spectrum2_index) in zip(hc_and_times.Spectrum, range(len(hc_and_times.Spectrum))):
    i=0
    while temp_log.get_value(i, col='Time') <= spectrum:
        i += 1   
        print(i)
    start_time = temp_log.get_value(i-1, col='Time')
    print(start_time)
    spectrum = hc_and_times.get_value(spectrum2_index+1, col='Spectrum')
    print(spectrum)
    a=i
    print(a)
    while temp_log.get_value(a, col='Time') <= spectrum:
        a += 1 
        print(a)
    finish_time = temp_log.get_value(a, col='Time')
    print(finish_time)
    if hc_and_times.get_value(spectrum2_index, col='hc') == 'c':
        temps = temp_log.cold[i:a]
    elif hc_and_times.get_value(spectrum2_index, col='hc') == 'NaN':  
        temps = [0,0,0]
    else:
        temps = temp_log.hot[i:a]
    temp_avg = np.average(temps)
    print(temp_avg)
    temps_avg.append(temp_avg)
    temp_std = np.std(temps)
    print(temp_std)
    temps_std.append(temp_std)
    

In [ ]:
spectra_indices = [1,2,71,72,141,142,211,212,281,283,347,348]
wavenums = np.array(raw_wavenumbers)

for spectrum in spectra_indices:       #plot planck curve at some temperature
    temp_avg = final_frame_4.at[spectrum,'temp_avg']
    std = final_frame_4.at[spectrum, 'temp_std']
    print(temp_avg)
    print(std)
    radiances_avg = []
    rad_minus = []
    rad_plus = []
    
    for wn in wavenums:
        radiances_avg.append(planck(wn, temp_avg+273.15))
        rad_minus.append(planck(wn, temp_avg+273.15-std))
        rad_plus.append(planck(wn, temp_avg+273.15+std))

    fig = plt.figure()
    plt.plot([x/100 for x in wavenums], [r*100 for r in radiances_avg]) #might need to *100 once we have dataframe back
    plt.plot([x/100 for x in wavenums], [r*100 for r in rad_minus])
    plt.plot([x/100 for x in wavenums], [r*100 for r in rad_plus])

    plt.title('Blackbody curve $\pm$ standard deviation for index ' + str(spectrum))
    plt.xlabel('wavenumber $cm^{-1}$')
    plt.ylabel('radiance $W m^{-2} sr^{-1} (cm^{-1})^{-1}$')
    plt.show()

In [ ]:
cut_spectra_times_4 = spectra_times_4[1:len(spectra_times_4)-(len(spectra_times_4)%10-1)] #list of times with assoc. response functions (starts from 1, until 2nd from last)

grouped_10_times = np.array(cut_spectra_times_4).reshape(int((len(cut_spectra_times_4))/10),10) #list length=35
beginning_10_times = [time[0] for time in grouped_10_times]

cut_max_frac_std = max_fractional_std[1:-1] #maximum fractional stds, divisible by 10 (remove first=cold and last=hot)
grouped_10_stds = np.array(cut_max_frac_std).reshape(int((len(cut_spectra_times_4))/10),10) #array length=35 of cut_max_frac_std grouped into 10
sum_sqs_10_stds = [sum(i**2 for i in grouped_10_stds[t]) for t in range(len(grouped_10_stds))] #sum of 10 values squared in grouped_10_stds
average_10_max_stds = [np.sqrt(i) for i in sum_sqs_10_stds] #list length=35, sum of all 10 std dev values, squared rooted (added in quadrature)

spectral_std_dev = temp_spectral_std_dev[1:-1] #list of radiance std devs for each of 350 spectra arrays over all 4001 wavelengths (remove first=cold, last=hot)
print(len(spectral_std_dev[0]))

#print(average_10_max_stds)
#print('')
#print(beginning_10_times)
#print('')
#print(spectra_times_4[0])
print(np.array(spectral_std_dev).shape)

#grouped_resp_fns = np.array(all_resp_functions).reshape(35,5,4001) #array dimensions 35,5,4001: 35 groups of 5 spectra with 4001 points each
#average_35_resp_fns = []
#plt.plot(wavenums,grouped_resp_fns[0][0])
#plt.plot(beginning_10_times, average_10_stds)

In [ ]:
# Plot standard deviation starting from the end to see at what point it stops fluctuating
# Take wavenums from 700 to 760 because they have a high response function

a = np.array(all_stand_devs).T.tolist()
#each element in a represents a wavenumber 

selected_stds = []
for (i,b) in zip(np.arange(len(wavenums)), wavenums):
    if 700<b<760:
        selected_stds.append(a[i])
        
print(len(selected_stds))

# to do this, would have to access the temp log again because the standard deviations above all include different numbers of temperature measurements

In [ ]:
# what period can we use for stability of measurements.
# pick range of wns and plot std from back pick range for strong response